In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import LSTM, Convolution1D, Flatten, Dropout, MaxPooling1D, Dense, Merge
# from keras.layers.convolution import 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import keras as kr
from keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight
from keras.optimizers import SGD,adam

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
print(kr.__version__,tf.__version__)

Using TensorFlow backend.


2.0.4 1.0.1


In [2]:
base_dir_tr='training/'
base_dir_vl='validation/'
weight_log='weight_logs/'

In [3]:
top_words = 20000
dtr_1=pd.read_csv(base_dir_tr+'data_manipulation_2_2_train.csv',encoding='ISO-8859-1')
dvl_1=pd.read_csv(base_dir_vl+'data_manipulation_2_2_valid.csv',encoding='ISO-8859-1')
dtr_2=pd.read_csv(base_dir_tr+'length_string_2.csv',encoding='ISO-8859-1')
dvl_2=pd.read_csv(base_dir_vl+'length_string_2.csv',encoding='ISO-8859-1')

In [4]:
dtr=pd.merge(dtr_1,dtr_2,on='sku_id',how='left')
Xl_tr=dtr[['len_title_des','len_short_des','price_rupiah_st']].as_matrix()
dvl=pd.merge(dvl_1,dvl_2,on='sku_id',how='left')
Xl_vl=dvl[['len_title_des','len_short_des','price_rupiah_st']].as_matrix()

In [5]:
Xsd=dtr.rank_short_des.as_matrix()
Xsd=np.array([ast.literal_eval(cd) for cd in Xsd])

Xti=dtr.rank_title.as_matrix()
Xti=np.array([ast.literal_eval(cd) for cd in Xti])

ycl=dtr_1.clari

In [6]:
Xsd_v=dvl.rank_short_des.as_matrix()
Xsd_v=np.array([ast.literal_eval(cd) for cd in Xsd_v])

Xti_v=dvl.rank_title.as_matrix()
Xti_v=np.array([ast.literal_eval(cd) for cd in Xti_v])

In [7]:
max_review_length_s = 150
max_review_length_t = 40
Xsd = sequence.pad_sequences(Xsd, maxlen=max_review_length_s)
Xti = sequence.pad_sequences(Xti, maxlen=max_review_length_t)

Xsd_v = sequence.pad_sequences(Xsd_v, maxlen=max_review_length_s)
Xti_v = sequence.pad_sequences(Xti_v, maxlen=max_review_length_t)

In [8]:
X=[]
for i,j,k in zip(Xsd,Xti,Xl_tr):
    X.append(list(k)+list(j)+list(i))
X=np.array(X)

X_v=[]
for i,j,k in zip(Xsd_v,Xti_v,Xl_vl):
    X_v.append(list(k)+list(j)+list(i))
X_v=np.array(X_v)

# np.savetxt(base_dir_vl+'data_input_ycl_3_1.txt',X_v)
np.savetxt(base_dir_vl+'data_input_ycl_5_1.txt',X_v)

In [9]:
len(X[0][3:])==len(np.array(list(Xti[0])+list(Xsd[0])))

True

In [10]:
len(X[0][:3])==len(np.array(list(Xl_tr[0])))

True

In [11]:
X[0][:3]

array([  7.00000000e+00,   1.50000000e+01,   5.20854665e-05])

#### ycl model

In [12]:
# model.save('model_yco_1_2_3.h5') OKE (74.05%)
ycl.value_counts()/ycl.value_counts().sum()

1    0.943362
0    0.056638
Name: clari, dtype: float64

In [13]:
clas_weight = class_weight.compute_class_weight('balanced', np.unique(ycl), ycl)
clas_weight

array([ 8.82798054,  0.53001928])

In [14]:
X_train, X_test, ycl_train, ycl_test = train_test_split(X, ycl, test_size=0.3, random_state=21)

X_train_nl=np.array([x[3:] for x in X_train])
X_train_ct=np.array([x[:3] for x in X_train])

X_test_nl=np.array([x[3:] for x in X_test])
X_test_ct=np.array([x[:3] for x in X_test])

In [15]:
ycl_test.value_counts()/ycl_test.value_counts().sum()

1    0.94497
0    0.05503
Name: clari, dtype: float64

In [16]:
# len(X_train_nl)
X_train

array([[  7.00000000e+00,   2.50000000e+01,   1.06085461e-04, ...,
          5.00000000e+01,   1.35000000e+02,   5.01300000e+03],
       [  1.30000000e+01,   2.00000000e+01,   2.56094165e-05, ...,
          1.73000000e+02,   6.00000000e+00,   2.50000000e+02],
       [  1.50000000e+01,   1.60000000e+01,   2.20754698e-05, ...,
          4.83100000e+03,   2.10000000e+01,   6.41000000e+02],
       ..., 
       [  1.50000000e+01,   2.20000000e+01,   6.80854648e-05, ...,
          1.53500000e+03,   2.33000000e+02,   1.17000000e+02],
       [  1.30000000e+01,   2.30000000e+01,   6.99788815e-05, ...,
          9.62000000e+02,   1.80000000e+01,   1.41220000e+04],
       [  1.20000000e+01,   1.50000000e+01,   8.20854633e-05, ...,
          2.55100000e+03,   3.30500000e+03,   0.00000000e+00]])

In [17]:
X_train_nl

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          5.00000000e+01,   1.35000000e+02,   5.01300000e+03],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.73000000e+02,   6.00000000e+00,   2.50000000e+02],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          4.83100000e+03,   2.10000000e+01,   6.41000000e+02],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.53500000e+03,   2.33000000e+02,   1.17000000e+02],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          9.62000000e+02,   1.80000000e+01,   1.41220000e+04],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          2.55100000e+03,   3.30500000e+03,   0.00000000e+00]])

In [22]:
np.random.seed(21)
#### Creating the model_1
embedding_vecor_length = 150
model_nl = Sequential()
model_nl.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))
# model_nl.add(Convolution1D(64, 4, border_mode='same'))
model_nl.add(LSTM(10))
# model_nl.add(Convolution1D(128, 3, border_mode='same'))
# model_nl.add(Convolution1D(64, 3, border_mode='same'))
# model_nl.add(Convolution1D(32, 3, border_mode='same'))
# model_nl.add(Convolution1D(16, 3, border_mode='same'))
# model_nl.add(Flatten())
model_nl.add(Dropout(0.1))
model_nl.add(Dense(100,activation='relu'))
model_nl.add(Dense(100,activation='relu'))
model_nl.add(Dense(100,activation='relu'))
# model_nl.add(Dense(150,activation='relu'))
model_nl.add(Dropout(0.1))
model_nl.add(Dense(2,activation='relu'))

#### Creating the model_2
model_ct = Sequential()
model_ct.add(Dense(50, input_shape=(3,),activation='relu'))
model_ct.add(Dense(50,activation='relu'))
# model_ct.add(Dense(20,activation='relu'))
model_ct.add(Dense(25,activation='relu'))
# model_ct.add(Dense(150,activation='relu'))
# model_ct.add(Dense(150,activation='relu'))
model_ct.add(Dropout(0.1))
model_ct.add(Dense(2,activation='relu'))

model = Sequential()
model.add(Merge([model_ct,model_nl], mode='concat'))
model.add(Dense(1,activation='sigmoid'))

model2=model

# opt = SGD(lr=0.1)
opt= adam(lr=0.0009) #0.001

# model.compile(loss = "categorical_crossentropy", optimizer = opt)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','mse'])
# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','mse'])
# model.compile(loss='mse', optimizer=opt, metrics=['accuracy','mse'])



# checkpoint
# filepath=weight_log+"weights_best_ycl_3_1.hdf5"
filepath=weight_log+"weights_best_ycl_5_1.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

C:\Users\Aska\Anaconda2\envs\snakes3_gpu\lib\site-packages\ipykernel\__main__.py:33: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [23]:
model.input_shape

[(None, 3), (None, 190)]

In [24]:
print(X_train_ct.shape,X_train_nl.shape)

(25398, 3) (25398, 190)


In [25]:
model.fit([X_train_ct,X_train_nl],ycl_train, class_weight=clas_weight, validation_data=([X_test_ct,X_test_nl],ycl_test),callbacks=callbacks_list, epochs=100, batch_size=100, verbose=1)

Train on 25398 samples, validate on 10885 samples
Epoch 1/100
25398/25398 [==============================] - 131s - loss: 0.2622 - acc: 0.9425 - mean_squared_error: 0.0602 - val_loss: 0.2355 - val_acc: 0.9450 - val_mean_squared_error: 0.0549
Epoch 2/100
25398/25398 [==============================] - 182s - loss: 0.2397 - acc: 0.9427 - mean_squared_error: 0.0566 - val_loss: 0.2220 - val_acc: 0.9450 - val_mean_squared_error: 0.0531
Epoch 3/100
25398/25398 [==============================] - 199s - loss: 0.2279 - acc: 0.9427 - mean_squared_error: 0.0551 - val_loss: 0.2129 - val_acc: 0.9450 - val_mean_squared_error: 0.0519
Epoch 4/100
25398/25398 [==============================] - 153s - loss: 0.2224 - acc: 0.9427 - mean_squared_error: 0.0543 - val_loss: 0.2113 - val_acc: 0.9450 - val_mean_squared_error: 0.0518
Epoch 5/100
25398/25398 [==============================] - 196s - loss: 0.2208 - acc: 0.9427 - mean_squared_error: 0.0543 - val_loss: 0.2116 - val_acc: 0.9450 - val_mean_squared_erro

KeyboardInterrupt: 

In [ ]:
# model2.load_weights(weight_log+"weights_best_ycl_3_1.hdf5")
model2.load_weights(weight_log+"weights_best_ycl_5_1.hdf5")

model2.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','mse'])

In [ ]:
#### Evaluation on the test set
ycl_predc=model2.predict_classes(X_test)
ycl_predp=model2.predict(X_test)

ycl_predc=[i[0] for i in ycl_predc]
ycl_predp=[i[0] for i in ycl_predp]

scores = model2.evaluate(X_test, ycl_test, verbose=0)
mse_c=mse(ycl_test,ycl_predc)
mse_p=mse(ycl_test,ycl_predp)
print("Accuracy: %.2f%%" % (scores[1]*100)) 
print("MSE_class : ",mse_c)
print("MSE_prob : ",mse_p)

In [49]:
# model.save('model_ycl_2_1_2.h5') <-3 epoch 0.04628
# model2.save('model_ycl_3_4.h5') #<-0.04567
model2.save('model_ycl_5_1.h5')

In [ ]:
#### model.save('model_ycl_3_3.h5')
np.random.seed(21)

# Using embedding from Keras
embedding_vecor_length = 150
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))

# model.add(Convolution1D(256, 3, border_mode='same'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Convolution1D(16, 3, border_mode='same'))
# model.add(Convolution1D(8, 3, border_mode='same'))
model.add(Flatten())
model.add(Dropout(0.35))
model.add(Dense(200,activation='sigmoid'))
model.add(Dense(200,activation='sigmoid'))
# model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

model2=model

# Log to tensorboard
# tensorBoardCallback = TensorBoard(log_dir='./logs', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])

### Validation

In [50]:
# X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_3_4.txt')
# ycl_model=load_model('model_ycl_3_4.h5')

X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_4_1.txt')
ycl_model=load_model('model_ycl_4_1.h5')

In [66]:
X_v==X_vl

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [51]:
ycl_predp=ycl_model.predict(X_vl)
ycl_predp=[i[0] for i in ycl_predp]

In [52]:
with open('clarity_valid.predict','w') as fcl:
    for pp in ycl_predp:
        fcl.write(str(pp))
        fcl.write('\n')
fcl.close()